In [1]:
import pandas as pd
import numpy as np
import sys
import jieba
import os
import pickle   #持久化
from numpy import *
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer #TF-IDF向量转换类
from sklearn.feature_extraction.text import TfidfVectorizer #TF_IDF向量生成类
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer 
from sklearn.datasets.base import Bunch
from sklearn.naive_bayes import MultinomialNB #多项式贝叶斯算法
import gc #解決memory error問題
from sklearn.utils import shuffle
import random
import gensim
from gensim.models import word2vec
import pickle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import np_utils
from tensorflow.keras import Sequential

Using TensorFlow backend.


In [2]:
with open('toutiao_cat_data.txt', 'r', encoding='utf8') as f:
    data = f.read().splitlines()

In [3]:
# data

In [4]:
label = []
title = []
keyword = []

for i in range(len(data)):
    label.append(data[i].split('_!_')[1])
    title.append(data[i].split('_!_')[3])
    keyword.append(data[i].split('_!_')[4])

In [5]:
keyword = pd.DataFrame(keyword)
title = pd.DataFrame(title)
label = pd.DataFrame(label)
trainData = pd.concat( [title , keyword], axis=1 )
trainData = pd.concat( [trainData , label], axis=1 )
trainData.columns = ['title', 'keyword', 'label']
trainData = shuffle(trainData)

# trainData = trainData[:20000]
trainData.index = range(len(trainData)) # row index從0開始
trainData = trainData.drop(['keyword'], axis=1)
trainData.head()

,title,label
0,王者荣耀到后期了，“泣血之刃”好还是“制裁之刃”好？,116
1,龙文4家企业入选省重点上市后备企业名单，看看有没有你熟悉的？,104
2,带金毛坐拖拉机，这货一脸不乐意，网友：没让你耕地就不错了！,115
3,文化人最能装：《金瓶梅》刻画得确实逼真！,101
4,如果奥创得逞，他能打过灭霸吗？,102


In [6]:
# #解決memory error問題，釋放記憶體
# del label
del title
del keyword
gc.collect()

55

In [7]:
#停用詞
stopwords=[]
for word in open('stopwords.txt','r', encoding='utf8'):
    stopwords.append(word.strip())

In [8]:
print(sys.getdefaultencoding())

utf-8


In [9]:
title_file = open("title_split.txt", "w", encoding = 'utf8')

#分詞(title和keyword)
splitTitle = []
for i in range(len(trainData.title)):
    split = jieba.cut(trainData.title[i], cut_all=False)
    temp = ""
    for word in split:
        if word not in stopwords:  #若word不在停用詞裡面，則加入
            temp = temp + word + " "
    temp = temp.strip()  #刪除不必要的空白
    
    title_file.writelines(str(temp + "\n"))   #存檔(分割詞，已扣除stopword)
    splitTitle.append(temp)

title_file.close()
    
# splitKeyword = []
# for i in range(len(trainData.keyword)):
#     split = jieba.cut(trainData.keyword[i], cut_all=False)
#     temp = ""
#     for word in split:
#         if word not in stopwords:  #若word不在停用詞裡面，則加入
#             temp = temp + word + " "   
#     temp = temp.strip()  #刪除不必要的空白
#     splitKeyword.append(temp)

Building prefix dict from the default dictionary ...
I0801 14:48:55.645139 12564 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jack\AppData\Local\Temp\jieba.cache
I0801 14:48:55.718110 12564 __init__.py:131] Loading model from cache C:\Users\jack\AppData\Local\Temp\jieba.cache
Loading model cost 0.924 seconds.
I0801 14:48:56.569814 12564 __init__.py:163] Loading model cost 0.924 seconds.
Prefix dict has been built succesfully.
I0801 14:48:56.569814 12564 __init__.py:164] Prefix dict has been built succesfully.


In [10]:
del stopwords
del split
del temp
gc.collect()

197

In [11]:
# TfidfTransformer + CountVectorizer  =  TfidfVectorizer

#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
# vectorizer=CountVectorizer() 
#该类会统计每个词语的tf-idf权值
# transformer=TfidfTransformer()   
#第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
# tfidf=transformer.fit_transform(vectorizer.fit_transform(splitTitle))
# word=vectorizer.get_feature_names()
# weight=tfidf.toarray()

# title_vectorizer = TfidfVectorizer(max_df = 0.5, min_df=2)
# title_tfidf = title_vectorizer.fit_transform(splitTitle)
# title_weight = title_tfidf.toarray()   #将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
# title_word = title_vectorizer.get_feature_names()   #获取词袋模型中的所有词语

# keyword_vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5)
# keyword_tfidf = keyword_vectorizer.fit_transform(splitTitle)
# keyword_weight = keyword_tfidf.toarray()
# keyword_word = keyword_vectorizer.get_feature_names()

In [12]:
# splitTitle

In [13]:
sentences = word2vec.LineSentence("title_split.txt")
model = word2vec.Word2Vec(sentences, size=300, min_count=1)
print(model)

Word2Vec(vocab=169098, size=300, alpha=0.025)


In [14]:
# for words in sentences:
#     print(words)

In [15]:
# 这里主要是要求得能代表文档的向量，这里就简单的将文档中的句子相加求平均，得到一个300维的文档向量
vecs = []
    
for words in sentences:
    # vec = np.zeros(2).reshape((1, 2))
    vec = np.zeros(300).reshape((1, 300))
    count = 0
    # words = remove_some(words)
    for word in words[1:]:
        try:
            count = count + 1
            # vec += model[word].reshape((1, 2))
            vec = vec + model[word].reshape((1, 300))
#             print(vec)
        except KeyError:
            continue
#     if count > 0:
    vec = vec/count
    vecs.append(vec)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [16]:
# 對dataset label 做編碼
le = LabelEncoder()
new_label = le.fit_transform(trainData.label).reshape(-1,1)

In [17]:
new_label.shape

(382688, 1)

In [18]:
# label list
new_label = new_label.tolist()

In [19]:
# # 3維轉2維
# vecs = np.reshape(vecs, (20000,300))
# print(vecs.shape)

In [20]:
temp = vecs.copy()
leng = len(temp)
index = 0
for i in range(leng):
    if np.isnan(temp[i]).any() == True:
        del vecs[index]
        del new_label[index]
        index = index - 1
        
    index = index + 1

In [21]:
# vecs = pd.DataFrame(vecs)
# vecs = vecs.fillna(vecs.mean())  # 把0補成平均值
# vecs = np.array(vecs, dtype='float64')
# print(vecs.shape)

In [22]:
# vecs = vecs[:,:,np.newaxis]
# print(vecs.shape)

In [23]:
np.isnan(vecs).any()

False

In [24]:
#要先轉乘np.array
new_label = np.array(new_label, dtype='float64')
# label encoding
new_label = np_utils.to_categorical(new_label,15)
print(new_label.shape)

(382308, 15)


In [25]:
#要先轉乘np.array
vecs = np.array(vecs, dtype='float64')
print(vecs.shape)

(382308, 1, 300)


In [26]:
# new_label = new_label[:,:,np.newaxis]
# new_label = new_label[:,np.newaxis,:]
# print(new_label.shape)

In [27]:
new_label[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
      dtype=float32)

In [28]:
# 随机选择20%作为测试集，剩余作为训练集
X_train, X_test, y_train, y_test = train_test_split(vecs, new_label, test_size=0.20, random_state=0)  

In [29]:
print(X_train[0])
print(X_test.shape)
print(y_train[0])
print(y_test.shape)

[[ 2.58074305e-01  1.58612642e-01 -1.44940097e-01  1.29025510e-01
   1.03847934e-01  1.28651097e-01  1.50737151e-01 -1.46670424e-01
   8.04184579e-02  7.52745291e-02 -2.90296446e-01 -5.14505968e-02
  -1.98715075e-01 -1.20859365e-01  2.72466362e-01 -1.25175389e-01
   5.38247213e-02  4.45781554e-02 -6.23296700e-02 -1.06773611e-01
   6.92078574e-02  1.52581002e-01  9.75786674e-02  3.41345274e-02
   4.84860047e-02  7.38384965e-02 -1.42878508e-01 -7.12984825e-02
  -1.08816757e-01 -1.19982400e-01  1.55660516e-01  3.89198248e-02
  -9.32216638e-02 -1.87144537e-01  1.15266062e-01  2.02732196e-01
   1.13264006e-02 -4.30892163e-02  8.55203599e-02  1.37583117e-02
   6.00714810e-02  1.02429857e-01 -2.64415396e-01 -1.34703120e-01
  -1.00851678e-01 -1.32131469e-01  3.03077609e-02  1.76636677e-01
  -1.70558867e-01  1.96760715e-01  7.79409857e-02 -1.54087906e-01
   1.04937453e-01 -1.13677219e-01 -2.26069128e-01 -4.13062514e-03
   7.85268647e-03 -5.30659871e-02  6.19218331e-02 -1.31557771e-01
  -2.30258

In [30]:
# clf = svm.SVC()  # 使用RBF核
# clf_res = clf.fit(X_train, y_train)

In [31]:
model = Sequential()

model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(32))
model.add(Dropout(0.2))

model.add(Dense(15, activation='softmax'))

model.summary()
model.compile(loss="categorical_crossentropy",optimizer='rmsprop',metrics=["accuracy"])

model_fit = model.fit(X_train,y_train,batch_size=100,epochs=10,
                      validation_data=(X_test,y_test),
#                      callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)] ## 當val-loss不再提升時停止訓練
                     )

W0801 14:52:00.408866 12564 deprecation.py:323] From D:\Anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 128)            219648    
_________________________________________________________________
dropout (Dropout)            (None, 1, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 15)                495       
Total params: 240,751
Trainable params: 240,751
Non-trainable params: 0
_________________________________________________________________
Train on 305846 samples, validate on 76462 samples
Epoch 1/10
305846/305846 [==============================] - 24s 7

In [32]:
# max_len = 100
# max_words = 5000

# ## LSTM模型
# inputs = Input(name='inputs',shape=[max_len])
# ## Embedding(詞彙表大小,batch大小,每個新聞的詞長)
# layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
# layer = LSTM(128)(layer)
# layer = Dense(128,activation="relu",name="FC1")(layer)
# layer = Dropout(0.5)(layer)
# layer = Dense(15,activation="softmax",name="FC2")(layer)

# model = Model(inputs=inputs,outputs=layer)
# model.summary()
# model.compile(loss="categorical_crossentropy",optimizer=RMSprop(),metrics=["accuracy"])

# model_fit = model.fit(X_train,y_train,batch_size=128,epochs=10,
#                       validation_data=(X_test,y_test),
#                       callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)] ## 當val-loss不再提升時停止訓練
#                      )

In [33]:
# # train_pred = clf_res.predict(X_train)
# test_pred = clf_res.predict(X_test)
# print(classification_report(y_test, test_pred))

In [34]:
test_pred = model.predict(X_test)
target_names = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']
# print(classification_report(y_test, test_pred))

In [35]:
test_pred[0]

array([8.6684922e-05, 8.9917833e-04, 1.4269979e-03, 6.2086654e-04,
       5.1810939e-02, 6.4793677e-04, 2.3059237e-03, 1.8959485e-03,
       9.3046373e-01, 2.3602888e-03, 3.0079446e-04, 3.6617669e-03,
       6.4990658e-05, 1.1413632e-03, 2.3125845e-03], dtype=float32)

In [36]:
result = model.evaluate(X_train, y_train)
print ('\ntrain Acc:', result[1])

305846/305846 [==============================] - 16s 52us/sample - loss: 0.8163 - accuracy: 0.7534

train Acc: 0.7533955


In [37]:
result = model.evaluate(X_test, y_test)
print ('\ntest Acc:', result[1])

76462/76462 [==============================] - 4s 54us/sample - loss: 0.8457 - accuracy: 0.7447

test Acc: 0.74469674
